Import Libraries and data

In [2]:
import importlib

import optuna
from scipy.sparse import csr_matrix
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from Optimize.SaveResults import SaveResults
from Optimize.slim import objective_function_SLIM
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.HybridOptunable import HybridOptunable
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import pandas as pd
import scipy.sparse as sps
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens1MReader import Movielens1MReader
from Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_SVDpp_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import Optimize
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Press Maiusc+F10 to execute it or replace it with your code.
# Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.


URM_all_dataframe = pd.read_csv(filepath_or_buffer="Data/data_train.csv",
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                engine='python')
users = pd.read_csv(filepath_or_buffer="Data/data_target_users_test.csv")

ICM = pd.read_csv(filepath_or_buffer="Data/data_ICM_metadata.csv",
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                engine='python')




n_interactions = len(URM_all_dataframe)
mapped_id, original_id = pd.factorize(URM_all_dataframe["user_id"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)
mapped_id, original_id = pd.factorize(URM_all_dataframe["item_id"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)
print(URM_all_dataframe['item_id'].max())
print(URM_all_dataframe.head(4))
#URM_all_dataframe["user_id"] = URM_all_dataframe["user_id"].map(user_original_ID_to_index)
#URM_all_dataframe["item_id"] = URM_all_dataframe["item_id"].map(item_original_ID_to_index)
print(URM_all_dataframe['item_id'].max())
print(URM_all_dataframe.head(4))
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
print(n_users, n_items)
URM_all = sps.coo_matrix((URM_all_dataframe["data"].values,
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)))
n_items = ICM["item_id"].max() + 1
n_features = ICM["feature_id"].max() + 1
ICM_all = sps.coo_matrix((ICM["data"].values, (ICM["item_id"].values, ICM["feature_id"].values)) , shape=(n_items, n_features))

URM_all.tocsr()
ICM_all.tocsr()

URM_trainval, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_trainval, train_percentage = 0.8)


C:\Users\leoli\miniconda3\envs\RecSysFramework\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
35736 38121


In [3]:
slimrec = SLIMElasticNetRecommender(URM_trainval)
slimrec.fit(alpha= 0.0002021210695683939, topK= 856, l1_ratio= 0.23722934371355184)

SLIMElasticNetRecommender: Processed 3036 ( 8.0%) in 5.00 min. Items per second: 10.11
SLIMElasticNetRecommender: Processed 6360 (16.7%) in 10.00 min. Items per second: 10.60
SLIMElasticNetRecommender: Processed 10180 (26.7%) in 15.00 min. Items per second: 11.31
SLIMElasticNetRecommender: Processed 13697 (35.9%) in 20.00 min. Items per second: 11.41
SLIMElasticNetRecommender: Processed 17583 (46.1%) in 25.00 min. Items per second: 11.72
SLIMElasticNetRecommender: Processed 21573 (56.6%) in 30.00 min. Items per second: 11.98
SLIMElasticNetRecommender: Processed 25087 (65.8%) in 35.00 min. Items per second: 11.94
SLIMElasticNetRecommender: Processed 28611 (75.1%) in 40.01 min. Items per second: 11.92
SLIMElasticNetRecommender: Processed 31115 (81.6%) in 45.01 min. Items per second: 11.52
SLIMElasticNetRecommender: Processed 34638 (90.9%) in 50.01 min. Items per second: 11.54
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 54.44 min. Items per second: 11.67


In [102]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, _ = evaluator_validation.evaluateRecommender(slimrec)

EvaluatorHoldout: Ignoring 151 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35585 (100.0%) in 29.00 sec. Users per second: 1227


In [103]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.087233,0.131671,0.113063,0.042766,0.063597,0.252069,0.126574,0.098483,0.509428,0.324933,...,0.995775,0.507276,0.995775,0.219391,13.303581,0.99984,0.363133,0.901585,1.468671,0.302643


In [6]:
graphrec = RP3betaRecommender(URM_trainval)
graphrec.fit(topK= 12, alpha= 0.5769111396825488, beta= 0.0019321798490027353)

RP3betaRecommender: Similarity column 38121 (100.0%), 3182.69 column/sec. Elapsed time 11.98 sec


EvaluatorHoldout: Processed 35295 (100.0%) in 23.88 sec. Users per second: 1478


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.071387,0.125528,0.113448,0.033362,0.057825,0.212056,0.11276,0.087632,0.451169,0.263824,...,0.98766,0.445601,0.98766,0.220129,13.330875,0.999858,0.364355,0.903435,1.396348,0.303685


In [104]:
from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender
evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

result_df, _ = evaluator_validation.evaluateRecommender(graphrec)

EvaluatorHoldout: Ignoring 151 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 35585 (100.0%) in 24.25 sec. Users per second: 1467


In [72]:
result_df

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.080037,0.121551,0.104794,0.038687,0.058017,0.232545,0.116348,0.090757,0.479359,0.296934,...,0.995775,0.477334,0.995775,0.210128,13.258731,0.999844,0.347801,0.898546,1.378108,0.303801


In [73]:
type(graphrec.W_sparse)

scipy.sparse.csr.csr_matrix

In [74]:
type(slimrec.W_sparse)

scipy.sparse.csr.csr_matrix

In [105]:
from Recommenders.HybridOptunable2 import HybridOptunable2


def  obj_hybrid(optuna_trial):
    print("helloworld")
    alpha = optuna_trial.suggest_float("alpha", 0.1, 0.9)
    recommender_object = HybridOptunable2(URM_trainval)
    recommender_object.fit(alpha,graphrec, slimrec )
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
    return result_df.loc[10][ "MAP"]


In [101]:
study = optuna.create_study(direction="maximize")
save_results = SaveResults()
study.optimize(obj_hybrid, callbacks= [save_results], n_trials=50)

[I 2024-11-28 00:16:59,810] A new study created in memory with name: no-name-1b070d56-dc73-45a6-999e-0f7c0cb8f92b


helloworld
EvaluatorHoldout: Processed 35585 (100.0%) in 29.46 sec. Users per second: 1208


[I 2024-11-28 00:17:29,427] Trial 0 finished with value: 0.05527284659355562 and parameters: {'alpha': 0.9916621698159258}. Best is trial 0 with value: 0.05527284659355562.


helloworld
EvaluatorHoldout: Processed 35585 (100.0%) in 37.57 sec. Users per second: 947


[I 2024-11-28 00:18:07,141] Trial 1 finished with value: 0.05526220139125661 and parameters: {'alpha': 0.9947163606909499}. Best is trial 0 with value: 0.05527284659355562.


helloworld


[W 2024-11-28 00:18:15,060] Trial 2 failed with parameters: {'alpha': 0.9906790234253808} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\leoli\miniconda3\envs\RecSysFramework\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\leoli\AppData\Local\Temp\ipykernel_47260\750185838.py", line 9, in obj_hybrid
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_object)
  File "C:\Users\leoli\OneDrive - Politecnico di Milano\primo anno mag\competition recsys\RecSysProject\Evaluation\Evaluator.py", line 276, in evaluateRecommender
    results_dict = self._run_evaluation_on_selected_users(recommender_object, self.users_to_evaluate)
  File "C:\Users\leoli\OneDrive - Politecnico di Milano\primo anno mag\competition recsys\RecSysProject\Evaluation\Evaluator.py", line 484, in _run_evaluation_on_selected_users
    results_dict = self._compute_metrics_on_re

KeyboardInterrupt: 

In [17]:
from main import evaluator_validation, save_results

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])
def obj_hybrid(optuna_trial):
    
    scoreshybridrecommender = ScoresHybridRecommender(URM_trainval, slimrec, graphrec)
    hyper={"alpha": optuna_trial.suggest_float("alpha", 0.4, 1)}
    scoreshybridrecommender.fit(hyper)
    result_df, _ = evaluator_validation.evaluateRecommender(scoreshybridrecommender)
    return result_df.loc[10, "MAP"]



38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
38120
   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
35736 38121
EvaluatorHoldout: Ignoring 170 ( 0.5%) Users that have less than 1 test interactions
SLIMElasticNetRecommender: Processed 2433 ( 6.4%) in 5.00 min. Items per second: 8.10
SLIMElasticNetRecommender: Processed 5426 (14.2%) in 10.00 min. Items per second: 9.04
SLIMElasticNetRecommender: Processed 8279 (21.7%) in 15.00 min. Items per second: 9.20
SLIMElasticNetRecommender: Processed 11093 (29.1%) in 20.01 min. Items per second: 9.24
SLIMElasticNetRecommender: Processed 14033 (36.8%) in 25.01 min. Items per second: 9.35
SLIMElasticNetRecommender: Processed 16818 (44.1%) in 30.01 min. Items per second: 9.34
SLIMElasticNetRecommender: Processed 19564 (51.3%) in 35.01 min. Items per second: 9.31
SLIMElasticNetRecom

EvaluatorHoldout: Processed 35295 (100.0%) in 39.13 sec. Users per second: 902


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.071421,0.125587,0.113463,0.0334,0.05792,0.212547,0.112895,0.087662,0.451027,0.264231,...,0.98766,0.445461,0.98766,0.219242,13.331373,0.999861,0.362887,0.903469,1.375212,0.303936
